<a href="https://colab.research.google.com/github/Abimbola-ai/Blood-Spectorscopy/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from prettytable import PrettyTable
from tqdm import tqdm
tqdm.pandas()
import tqdm
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')
seed = 42

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  after removing the cwd from sys.path.


In [2]:
# train = pd.read_csv("Data/clean_train.csv",index_col=[0])
# test = pd.read_csv("Data/clean_test.csv",index_col=[0])
train = pd.read_csv("clean_train.csv",index_col=[0])
test = pd.read_csv("clean_test.csv",index_col=[0])

In [3]:
scaled_features_train = train.copy()
scaled_features_test = test.copy()

def scale_data(df):
    col_names = ['absorbance0', 'absorbance1', 'absorbance2',
       'absorbance3', 'absorbance4', 'absorbance5', 'absorbance6',
       'absorbance7', 'absorbance8', 'absorbance9', 'absorbance10',
       'absorbance11', 'absorbance12', 'absorbance13', 'absorbance14',
       'absorbance15', 'absorbance16', 'absorbance17', 'absorbance18',
       'absorbance19', 'absorbance20', 'absorbance21', 'absorbance22',
       'absorbance23', 'absorbance24', 'absorbance25', 'absorbance26',
       'absorbance27', 'absorbance28', 'absorbance29', 'absorbance30',
       'absorbance31', 'absorbance32', 'absorbance33', 'absorbance34',
       'absorbance35', 'absorbance36', 'absorbance37', 'absorbance38',
       'absorbance39', 'absorbance40', 'absorbance41', 'absorbance42',
       'absorbance43', 'absorbance44', 'absorbance45', 'absorbance46',
       'absorbance47', 'absorbance48',
       'absorbance49', 'absorbance50', 'absorbance51', 'absorbance52',
       'absorbance53', 'absorbance54', 'absorbance55', 'absorbance56',
       'absorbance57', 'absorbance58', 'absorbance59', 'absorbance60',
       'absorbance61', 'absorbance62', 'absorbance63', 'absorbance64',
       'absorbance65', 'absorbance66', 'absorbance67', 'absorbance68',
       'absorbance69', 'absorbance70', 'absorbance71', 'absorbance72',
       'absorbance73', 'absorbance74', 'absorbance75', 'absorbance76',
       'absorbance77', 'absorbance78', 'absorbance79', 'absorbance80',
       'absorbance81', 'absorbance82', 'absorbance83', 'absorbance84',
       'absorbance85', 'absorbance86', 'absorbance87', 'absorbance88',
       'absorbance89', 'absorbance90', 'absorbance91', 'absorbance92',
       'absorbance93', 'absorbance94', 'absorbance95', 'absorbance96',
       'absorbance97', 'absorbance98',
       'absorbance99', 'absorbance100', 'absorbance101', 'absorbance102',
       'absorbance103', 'absorbance104', 'absorbance105', 'absorbance106',
       'absorbance107', 'absorbance108', 'absorbance109', 'absorbance110',
       'absorbance111', 'absorbance112', 'absorbance113', 'absorbance114',
       'absorbance115', 'absorbance116', 'absorbance117', 'absorbance118',
       'absorbance119', 'absorbance120', 'absorbance121', 'absorbance122',
       'absorbance123', 'absorbance124', 'absorbance125', 'absorbance126',
       'absorbance127', 'absorbance128', 'absorbance129', 'absorbance130',
       'absorbance131', 'absorbance132', 'absorbance133', 'absorbance134',
       'absorbance135', 'absorbance136', 'absorbance137', 'absorbance138',
       'absorbance139', 'absorbance140', 'absorbance141', 'absorbance142',
       'absorbance143', 'absorbance144', 'absorbance145', 'absorbance146',
       'absorbance147', 'absorbance148', 'absorbance149', 'absorbance150',
       'absorbance151', 'absorbance152', 'absorbance153', 'absorbance154',
       'absorbance155', 'absorbance156', 'absorbance157', 'absorbance158',
       'absorbance159', 'absorbance160', 'absorbance161', 'absorbance162',
       'absorbance163', 'absorbance164', 'absorbance165', 'absorbance166',
       'absorbance167', 'absorbance168', 'absorbance169', 'temperature',
       'humidity','std', 'low_abs', 'high_abs', 'med_abs',
       'mean_abs', 'std_abs', 'sum_abs']
    features = df[col_names]
    scaler = MinMaxScaler().fit(features.values)
    features = scaler.transform(features.values)
    df[col_names] = features
    return df

In [4]:
train_scaled = scale_data(scaled_features_train)
test_scaled = scale_data(scaled_features_test)

In [5]:
train_scaled.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
0,1975,0.652042,0.642710,0.633364,0.623264,0.622399,0.609421,0.592866,0.584126,0.577179,0.584544,0.584612,0.575608,0.563661,0.566091,0.574274,0.574363,0.568399,0.559546,0.550318,0.544750,0.539488,0.534500,0.531351,0.532029,0.527798,0.527859,0.523982,0.523038,0.522653,0.524182,0.523920,0.523094,0.521996,0.524124,0.525372,0.524978,0.526299,0.529976,0.532574,...,0.836378,0.840233,0.839506,0.839962,0.839219,0.837026,0.837737,0.843035,0.848269,0.843045,0.844892,0.841455,0.842998,0.850207,0.846778,0.848576,0.845237,0.845864,0.854340,0.856793,0.848577,0.857827,0.836894,0.831730,0.802424,0.775454,0.755963,0.722586,0.478599,0.504649,0,1,1,0.790323,0.654070,0.817091,0.633388,0.781237,0.805277,0.781237
1,1982,0.698607,0.687373,0.673512,0.669837,0.664034,0.648212,0.627822,0.618668,0.610307,0.610356,0.606375,0.592257,0.580937,0.582689,0.589071,0.591793,0.589665,0.582626,0.574444,0.570193,0.567231,0.562530,0.560525,0.561526,0.557320,0.557500,0.554911,0.553563,0.551274,0.551599,0.549970,0.546978,0.547670,0.545700,0.544923,0.543503,0.542914,0.544344,0.542953,...,0.464522,0.464833,0.463970,0.465898,0.468244,0.463599,0.466964,0.468712,0.476127,0.474260,0.473102,0.473570,0.477840,0.480206,0.482531,0.480070,0.483585,0.484095,0.496865,0.506689,0.525256,0.566295,0.592109,0.613829,0.608543,0.602292,0.574935,0.560630,0.328331,0.640465,0,1,2,0.306452,0.700665,0.516564,0.479843,0.519043,0.472824,0.519043
2,1989,0.680759,0.676126,0.666238,0.668406,0.669962,0.662563,0.643781,0.629202,0.610788,0.609148,0.603838,0.588867,0.574873,0.571789,0.575252,0.578197,0.574625,0.564417,0.555309,0.548720,0.541367,0.537955,0.534872,0.536361,0.532451,0.532536,0.531770,0.529857,0.529777,0.530688,0.529730,0.528411,0.529555,0.529442,0.529291,0.528634,0.528365,0.529740,0.530062,...,0.535834,0.531428,0.523257,0.521788,0.516523,0.511554,0.509964,0.514012,0.508708,0.503256,0.503507,0.501108,0.497951,0.502847,0.499659,0.497630,0.493270,0.496097,0.497241,0.491652,0.481675,0.487550,0.466362,0.454566,0.426858,0.416660,0.400924,0.385907,0.484870,0.719316,0,1,2,0.354839,0.682876,0.750031,0.512484,0.612633,0.653710,0.612633
3,2018,0.674998,0.668067,0.660314,0.658546,0.652947,0.650522,0.640554,0.632638,0.623293,0.616307,0.611317,0.607453,0.596010,0.596676,0.599148,0.599165,0.602098,0.601598,0.597846,0.595783,0.593897,0.591960,0.588053,0.590839,0.584312,0.584694,0.583574,0.583560,0.580798,0.581533,0.579724,0.578268,0.578281,0.576649,0.574575,0.574457,0.571222,0.572198,0.571567,...,0.718847,0.720803,0.725154,0.727057,0.728323,0.738098,0.730499,0.736235,0.744700,0.746553,0.745761,0.750463,0.752849,0.746514,0.754540,0.749180,0.759979,0.752283,0.758489,0.763540,0.767606,0.783928,0.785280,0.785107,0.775524,0.760129,0.770775,0.769760,0.369968,0.739670,2,1,1,0.935484,0.677097,0.724927,0.572940,0.705100,0.685085,0.705100
4,2045,0.568017,0.557618,0.541519,0.530828,0.526682,0.510204,0.493224,0.482760,0.467903,0.455089,0.451552,0.448340,0.439008,0.434328,0.429546,0.42467

In [6]:
train.head()

,donation_id,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,std,low_abs,high_abs,med_abs,mean_abs,std_abs,sum_abs
0,1975,0.505055,0.506570,0.511544,0.516300,0.524067,0.530191,0.536138,0.542317,0.551118,0.567411,0.582043,0.594632,0.601357,0.606002,0.610174,0.610535,0.607241,0.602593,0.598158,0.594082,0.589946,0.586214,0.582763,0.579071,0.576070,0.572933,0.569674,0.567799,0.566049,0.564866,0.564235,0.563938,0.564077,0.565843,0.567793,0.570145,0.573502,0.578465,0.583251,...,1.551025,1.539729,1.531069,1.514383,1.502906,1.490929,1.480540,1.471343,1.463302,1.453560,1.445046,1.434967,1.428055,1.423711,1.417191,1.414312,1.410027,1.409971,1.410790,1.410953,1.410468,1.409675,1.401169,1.395687,1.390813,1.384304,1.395545,1.369307,42.332500,37.773000,0,1,1,0.0073,0.505055,1.927817,0.958842,1.135224,0.497048,192.988096
1,1982,0.523531,0.523485,0.526291,0.532875,0.538397,0.543180,0.547520,0.553433,0.561674,0.575610,0.588949,0.599821,0.606795,0.611225,0.614827,0.615984,0.613867,0.609749,0.605598,0.601913,0.598477,0.594828,0.591709,0.588105,0.585093,0.581967,0.579102,0.577092,0.574750,0.573190,0.572159,0.571213,0.571900,0.572428,0.573744,0.575815,0.578596,0.582865,0.586430,...,1.409270,1.398967,1.391729,1.378697,1.369708,1.358800,1.350754,1.342818,1.337081,1.329671,1.321694,1.314480,1.309197,1.304323,1.300707,1.296805,1.294782,1.293546,1.295560,1.296753,1.298688,1.301346,1.299807,1.299004,1.298251,1.297575,1.302398,1.284716,39.385167,43.316667,0,1,2,0.0043,0.523485,1.734845,0.916559,1.061186,0.421130,180.401567
2,1989,0.516449,0.519226,0.523619,0.532366,0.540438,0.547985,0.552716,0.556823,0.561827,0.575226,0.588143,0.598764,0.604886,0.607795,0.610482,0.611734,0.609181,0.604104,0.599697,0.595304,0.590524,0.587276,0.583843,0.580398,0.577492,0.574359,0.572048,0.569875,0.568215,0.566841,0.566002,0.565557,0.566380,0.567466,0.568986,0.571264,0.574136,0.578393,0.582481,...,1.436455,1.423938,1.413727,1.398970,1.387043,1.375768,1.365806,1.358372,1.348132,1.339412,1.331781,1.323498,1.315743,1.311629,1.306184,1.302404,1.297868,1.297409,1.295682,1.291848,1.283621,1.272086,1.247736,1.228340,1.211511,1.204599,1.212862,1.193458,42.455500,46.535167,0,1,2,0.0046,0.516449,1.884757,0.925547,1.087614,0.462437,184.894345
3,2018,0.514163,0.516174,0.521443,0.528857,0.534581,0.543953,0.551666,0.557929,0.565811,0.577500,0.590517,0.604558,0.611540,0.615626,0.617995,0.618288,0.617740,0.615631,0.612815,0.609790,0.606677,0.603872,0.600150,0.597083,0.593344,0.590255,0.587839,0.586225,0.583725,0.582278,0.581210,0.580745,0.581227,0.581873,0.582770,0.585287,0.587275,0.591395,0.595196,...,1.506221,1.494947,1.488640,1.473428,1.463089,1.455925,1.443002,1.434673,1.428175,1.421145,1.412157,1.405165,1.398711,1.390253,1.387694,1.382617,1.382858,1.379854,1.379893,1.380534,1.382475,1.382215,1.379796,1.375001,1.377971,1.376628,1.403167,1.393946,40.201833,47.366000,2,1,1,0.0082,0.514163,1.868637,0.942196,1.113725,0.469601,189.333202
4,2045,0.471717,0.474344,0.477808,0.483401,0.491121,0.496970,0.503694,0.509696,0.516301,0.526289,0.539822,0.554961,0.562119,0.564544,0.564670

In [7]:
# Raw Data
y_hdl_cholesterol_human = train_scaled['hdl_cholesterol_human']
y_hemoglobin_hgb_human = train_scaled['hemoglobin(hgb)_human']
y_cholesterol_ldl_human = train_scaled["cholesterol_ldl_human"]
train_unscaled = train.drop(['donation_id', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human'], axis=1)
train_scaled_drop = train_scaled.drop(['donation_id', 'hdl_cholesterol_human', 'hemoglobin(hgb)_human',
       'cholesterol_ldl_human'], axis=1)

df_train = train_scaled_drop.copy()
df_trainu = train_unscaled.copy() #Unscaled data
print(df_train.shape)
print(y_hdl_cholesterol_human.shape)
print(y_hemoglobin_hgb_human.shape)
print(y_cholesterol_ldl_human.shape)

(355, 179)
(355,)
(355,)
(355,)


In [8]:
X_cols = list(df_train.columns)
X_test = test_scaled[X_cols] # Scaled test data
X_testu = test[X_cols] # Unscaled test data

In [9]:
def hyperparameter_model(models, params, x_train, y_train):
    '''
    Hyperparameter tuning with RepeatedStratifiedKFold follow by GridSearchCV
    
    Parameters:
    models: Instance of the model
    params: list of parameters with value for tuning (dict)
    
    Return:
    grid_clf: return gridsearch model    
    '''
    str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=seed)
    grid_clf = GridSearchCV(models, params, cv=str_cv, return_train_score=True, scoring='roc_auc')
    grid_clf.fit(x_train, y_train)
    return grid_clf

def forward_selection_model(x_train, y_train,model, top_n=10):
    top_column = []
    exist_score = 0
    for n in range(top_n):
        print('for {} feature'.format(n+1))
        flag = 0
        for i in tqdm.tqdm_notebook(range(X_test.shape[1])):
            if len(top_column) == 0:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,[i]],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
            elif i not in top_column:
                str_cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
                score = cross_val_score(model,x_train[:,np.concatenate((top_column,[i]))],y_train,cv=str_cv,scoring='roc_auc')
                if exist_score < np.mean(score):
                    top_current = i
                    exist_score = np.mean(score)
                    flag = 1
        
        if flag == 1:
            print('Current top feature {} and score: {}'.format(top_current,exist_score))
            print('Appended to top column')
            top_column.append(top_current)
            print(top_column)
        else:
            break
    return top_column


def plot_feature_importance(model, model_name, top_n = 10):
    '''  
    Parameters:
    model: model 
    model_name: name of the model
  
    Return:
    Top 10 features of the dataset  
    '''       
    column_name = train.drop(['id','target'], axis=1).columns
    if model_name == 'log_model':
        feat_imp_coef = model.coef_.ravel()
    else:
        feat_imp_coef = model.feature_importances_
    temp = pd.DataFrame(data=np.column_stack((column_name, feat_imp_coef)), columns=['col_name','coef'])
    temp = temp.sort_values(by='coef', ascending=False).reset_index()
    df = temp
    temp = temp[:top_n]
    return temp

In [10]:
reverse_mapping = {'0' : 'low', '1' : 'ok', '2' : 'high'}

#### Build each model individually

##### A. Baseline Model - Logistic Regression

- hdl cholesterol human

In [11]:
# log_reg = LogisticRegression(random_state=seed)
# log_reg.fit(df_train,y_hdl_cholesterol_human)
# hdl_cholesterol_human = log_reg.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

- ldl cholesterol human

In [12]:
# log_reg2 = LogisticRegression(random_state=seed)
# log_reg2.fit(df_train,y_cholesterol_ldl_human)
# cholesterol_ldl_human = log_reg2.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

- hemoglobin human

In [13]:
# log_reg3 = LogisticRegression(random_state=seed)
# log_reg3.fit(df_train,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = log_reg3.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Leaderboard score with logistic regression is 0.5821917808219178

##### B. Logistic Regression with optimization

- hdl cholesterol human

In [14]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_hdl = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_hdl = hyperparameter_model(log_model_hdl, params, df_train,y_hdl_cholesterol_human)
# print(log_clf_hdl.best_params_)

# log_model_hdl = LogisticRegression(**log_clf_hdl.best_params_, class_weight='balanced', random_state=42)
# log_model_hdl.fit(df_train,y_hdl_cholesterol_human)

# hdl_cholesterol_human = log_model_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

- ldl cholesterol human

In [15]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_ldl = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_ldl = hyperparameter_model(log_model_ldl, params, df_train,y_cholesterol_ldl_human)
# print(log_clf_ldl.best_params_)

# log_model_ldl = LogisticRegression(**log_clf_ldl.best_params_, class_weight='balanced', random_state=seed)
# log_model_ldl.fit(df_train,y_cholesterol_ldl_human)

# cholesterol_ldl_human = log_model_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

- hemoglobin human

In [16]:
# params = {'penalty':['l1', 'l2', 'elasticnet'], 'C':[10**i for i in range(-4,5)], 'solver':['liblinear','sag']}
# log_model_hgb = LogisticRegression(random_state=42, class_weight='balanced')
# log_clf_hgb = hyperparameter_model(log_model_hgb, params, df_train,y_hemoglobin_hgb_human)
# print(log_clf_hgb.best_params_)

# log_model_hgb = LogisticRegression(**log_clf_hgb.best_params_, class_weight='balanced', random_state=seed)
# log_model_hgb.fit(df_train,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = log_model_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Submission score dropped to 0.5753424657534246 

#### C. PCA & Logistic Regression

In [17]:
# # # https://scikit-learn.org/stable/auto_examples/compose/plot_digits_pipe.html
# # # https://machinelearningmastery.com/multinomial-logistic-regression-with-python/

# from sklearn.decomposition import PCA
# from sklearn.pipeline import Pipeline
# def pca_lr(x_train, y_train):
#     pca = PCA()

#     logistic = LogisticRegression(random_state = 42,solver='liblinear',max_iter=10000, tol=0.1)
#     pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])


# # Parameters of pipelines can be set using ‘__’ separated parameter names:
#     param_grid = {
#     'pca__n_components': np.arange(170),
#     'logistic__C': np.logspace(-4, 4, 4),
#     }
#     search = GridSearchCV(pipe, param_grid, n_jobs=-1)
#     search.fit(x_train, y_train)
#     print('Best parameter (CV score=%0.3f):' % search.best_score_)
#     print(search.best_params_)

#     # Plot the PCA spectrum
#     pca.fit(x_train)

#     fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
#     ax0.plot(np.arange(1, pca.n_components_ + 1),
#          np.cumsum(pca.explained_variance_ratio_), '+', linewidth=2)
#     ax0.set_ylabel('PCA cummulative explained variance ratio')

#     ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
#             linestyle=':', label='n_components chosen')
#     ax0.legend(prop=dict(size=12))

#     # For each number of components, find the best classifier results
#     results = pd.DataFrame(search.cv_results_)
#     components_col = 'param_pca__n_components'
#     best_clfs = results.groupby(components_col).apply(
#     lambda g: g.nlargest(1, 'mean_test_score'))

#     best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
#                legend=False, ax=ax1)
#     ax1.set_ylabel('Classification accuracy (val)')
#     ax1.set_xlabel('n_components')

#     plt.xlim(-1, 70)
#     plt.tight_layout()
#     plt.show()

#     #print(f'Cummulative explained variance ratio:',np.cumsum(pca.explained_variance_ratio_))


- hdl cholesterol human

In [18]:
# pca_lr(df_train,y_hdl_cholesterol_human)
# hdl_cholesterol_human = search.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [19]:
# pca_lr(df_train,y_cholesterol_ldl_human)
# cholesterol_ldl_human = search.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [20]:
# pca_lr(df_train,y_hemoglobin_hgb_human)
# hemoglobin_hgb_human = search.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Score with PCA is 0.56 

#### D. Random Forest Classifier

In [21]:
# from sklearn.ensemble import RandomForestClassifier

# rf_hdl = RandomForestClassifier(random_state=seed)
# rf_hdl.fit(df_train,y_hdl_cholesterol_human)
# print("Random forest trained on unscaled data")

# hdl_cholesterol_human = rf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()


In [22]:
# rf_ldl = RandomForestClassifier(random_state=seed)
# rf_ldl.fit(df_train,y_cholesterol_ldl_human)
# print("Random forest trained on unscaled data")

# cholesterol_ldl_human = rf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [23]:
# rf_hgb = RandomForestClassifier(random_state=seed)
# rf_hgb.fit(df_train,y_hemoglobin_hgb_human)
# print("Random forest trained on unscaled data")

# hemoglobin_hgb_human = rf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

RF without optimization gave a score of 0.541 with unscaled data. and 0.51 with scaled data.

#### E. Random Forest Classifier with optimization

In [24]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_ldl = RandomForestClassifier(random_state=seed)
# rf_clf_ldl = hyperparameter_model(rf_model_ldl, params,df_train,y_cholesterol_ldl_human)
# print(rf_clf_ldl.best_params_)
# rf_clf_ldl = RandomForestClassifier(**rf_clf_ldl.best_params_,random_state=seed)
# rf_clf_ldl.fit(df_trainu,y_cholesterol_ldl_human)

# cholesterol_ldl_human = rf_clf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [25]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_hdl = RandomForestClassifier(random_state=seed)
# rf_clf_hdl = hyperparameter_model(rf_model_hdl, params,df_train,y_hdl_cholesterol_human)
# print(rf_clf_hdl.best_params_)
# rf_clf_hdl = RandomForestClassifier(**rf_clf_hdl.best_params_,random_state=seed)
# rf_clf_hdl.fit(df_trainu,y_hdl_cholesterol_human)

# hdl_cholesterol_human = rf_clf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [26]:
# params = {'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}
# rf_model_hgb = RandomForestClassifier(random_state=seed)
# rf_clf_hgb = hyperparameter_model(rf_model_hgb, params,df_train,y_hemoglobin_hgb_human)
# print(rf_clf_hgb.best_params_)
# rf_clf_hgb = RandomForestClassifier(**rf_clf_hgb.best_params_,random_state=seed)
# rf_clf_hgb.fit(df_trainu,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = rf_clf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

Rf with optimization on unscaled data gives 0.582 score, with scaled data gies 0.554

#### F. SVC Classifier 

In [27]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_ldl = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_ldl = hyperparameter_model(svc_model_ldl, params,df_train,y_cholesterol_ldl_human)
# print(svc_clf_ldl.best_params_)
# svc_clf_ldl = SVC(**svc_clf_ldl.best_params_,random_state=seed,probability=True)
# svc_clf_ldl.fit(df_trainu,y_cholesterol_ldl_human)

# cholesterol_ldl_human = rf_clf_ldl.predict(X_test)
# test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
# test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
# test['cholesterol_ldl_human'].value_counts()

In [28]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_hdl = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_hdl = hyperparameter_model(svc_model_hdl, params,df_train,y_hdl_cholesterol_human)
# print(svc_clf_hdl.best_params_)
# svc_clf_hdl = SVC(**svc_clf_hdl.best_params_,random_state=seed,probability=True)
# svc_clf_hdl.fit(df_trainu,y_hdl_cholesterol_human)

# hdl_cholesterol_human = rf_clf_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [29]:
# params = {'C':[10**i for i in range(-4,5)], 'kernel':['linear','poly','sigmoid','rbf']}
# svc_model_hgb = SVC(class_weight='balanced', random_state=seed, probability=True)
# svc_clf_hgb = hyperparameter_model(svc_model_hgb, params,df_train,y_hemoglobin_hgb_human)
# print(svc_clf_hgb.best_params_)
# svc_clf_hgb = SVC(**svc_clf_hgb.best_params_,random_state=seed,probability=True)
# svc_clf_hgb.fit(df_trainu,y_hemoglobin_hgb_human)

# hemoglobin_hgb_human = svc_clf_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

With unscaled data gave a score of 0.5829

##### G. Catboost Algorithm
https://journalofbigdata.springeropen.com/articles/10.1186/s40537-020-00349-y

- hdl cholesterol human

In [30]:
!pip install catboost

In [31]:
!pip install optuna


In [32]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import optuna

def objective(trial):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    param = {
        "loss_function": trial.suggest_categorical("loss_function", ["MultiClass","MultiClassOneVsAll"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),  
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    reg = CatBoostClassifier(**param)
    reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0, early_stopping_rounds=100)
    y_pred = reg.predict(X_test)
    score = accuracy_score(y_test, y_pred,normalize=False)
    return score

In [33]:
# "Logloss", "CrossEntropy", "MultiClassOneVsAll",

In [34]:
#LDL
from optuna.samplers import TPESampler

X = df_train
y = y_cholesterol_ldl_human

study = optuna.create_study(sampler=TPESampler(), direction="maximize")
study.optimize(objective, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-03-03 13:09:02,127] A new study created in memory with name: no-name-c1e0304b-8268-4df4-a1c0-052bb3934f18
[I 2022-03-03 13:09:17,187] Trial 0 finished with value: 64.0 and parameters: {'loss_function': 'MultiClassOneVsAll', 'learning_rate': 0.0005082181545580776, 'l2_leaf_reg': 0.02434332224091455, 'colsample_bylevel': 0.07695581371024932, 'depth': 5, 'boosting_type': 'Plain', 'bootstrap_type': 'MVS', 'min_data_in_leaf': 15, 'one_hot_max_size': 7}. Best is trial 0 with value: 64.0.
[I 2022-03-03 13:09:26,193] Trial 1 finished with value: 63.0 and parameters: {'loss_function': 'MultiClassOneVsAll', 'learning_rate': 8.266839251609268e-05, 'l2_leaf_reg': 0.2038192430444397, 'colsample_bylevel': 0.08936340701373298, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'min_data_in_leaf': 3, 'one_hot_max_size': 12, 'bagging_temperature': 7.935061333243446}. Best is trial 0 with value: 64.0.
[I 2022-03-03 13:09:37,525] Trial 2 finished with value: 48.0 and parameters: 

Number of completed trials: 10
Best trial:
	Best Score: 64.0
	Best Params: 
    loss_function: MultiClassOneVsAll
    learning_rate: 0.0005082181545580776
    l2_leaf_reg: 0.02434332224091455
    colsample_bylevel: 0.07695581371024932
    depth: 5
    boosting_type: Plain
    bootstrap_type: MVS
    min_data_in_leaf: 15
    one_hot_max_size: 7


In [35]:
# cat_model_ldl = CatBoostClassifier(loss_function = 'MultiClass',
#                                    learning_rate = 6.447911714090859e-05,
#                                    l2_leaf_reg = 0.02120959816527062,colsample_bylevel = 0.02345568909821258,depth = 3,
#                                    boosting_type = "Plain",bootstrap_type = "Bayesian",min_data_in_leaf = 3,
#                                    one_hot_max_size = 11,bagging_temperature = 3.8517108825279402,)

In [38]:
cat_model_ldl = CatBoostClassifier(loss_function = 'MultiClassOneVsAll',
                                   learning_rate = 0.0005082181545580776,
                                   l2_leaf_reg = 0.02434332224091455,colsample_bylevel = 0.07695581371024932,depth = 5,
                                   boosting_type = "Plain",bootstrap_type = "MVS",min_data_in_leaf = 15,
                                   one_hot_max_size = 7)

In [39]:
# Cholesterol ldl
# cat_ldf = cat_model_boost(df_train,y_cholesterol_ldl_human)
cat_ldl = cat_model_ldl.fit(df_train,y_cholesterol_ldl_human)
cholesterol_ldl_human = cat_ldl.predict(X_test)
test['cholesterol_ldl_human'] = cholesterol_ldl_human.astype(str)
test['cholesterol_ldl_human'] = test['cholesterol_ldl_human'].replace(reverse_mapping)
test['cholesterol_ldl_human'].value_counts()

0:	learn: 0.6929750	total: 7.41ms	remaining: 7.4s
1:	learn: 0.6928107	total: 15.7ms	remaining: 7.86s
2:	learn: 0.6926521	total: 23.8ms	remaining: 7.92s
3:	learn: 0.6924875	total: 32ms	remaining: 7.98s
4:	learn: 0.6923238	total: 41.1ms	remaining: 8.18s
5:	learn: 0.6921685	total: 48.2ms	remaining: 7.99s
6:	learn: 0.6920020	total: 57.9ms	remaining: 8.21s
7:	learn: 0.6918330	total: 66.7ms	remaining: 8.27s
8:	learn: 0.6916768	total: 74.2ms	remaining: 8.18s
9:	learn: 0.6915135	total: 82.2ms	remaining: 8.13s
10:	learn: 0.6913486	total: 90.4ms	remaining: 8.13s
11:	learn: 0.6911875	total: 98.2ms	remaining: 8.08s
12:	learn: 0.6910057	total: 105ms	remaining: 7.99s
13:	learn: 0.6908458	total: 115ms	remaining: 8.07s
14:	learn: 0.6906753	total: 122ms	remaining: 8.02s
15:	learn: 0.6905136	total: 131ms	remaining: 8.07s
16:	learn: 0.6903526	total: 139ms	remaining: 8.03s
17:	learn: 0.6901833	total: 146ms	remaining: 7.98s
18:	learn: 0.6900284	total: 154ms	remaining: 7.97s
19:	learn: 0.6898644	total: 165m

ok      56
high     5
Name: cholesterol_ldl_human, dtype: int64

In [40]:
# HDL Cholesterol
X = df_train
y = y_hdl_cholesterol_human

study = optuna.create_study(sampler=TPESampler(), direction="maximize")
study.optimize(objective, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-03-03 13:14:45,091] A new study created in memory with name: no-name-065a1fdc-b4cf-4558-9aa6-b675288d9f8e
[I 2022-03-03 13:14:45,468] Trial 0 finished with value: 55.0 and parameters: {'loss_function': 'MultiClassOneVsAll', 'learning_rate': 0.014118274549152285, 'l2_leaf_reg': 0.49651331089079853, 'colsample_bylevel': 0.030520351783126252, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 20, 'one_hot_max_size': 8, 'subsample': 0.8295901353832188}. Best is trial 0 with value: 55.0.
[I 2022-03-03 13:15:51,229] Trial 1 finished with value: 52.0 and parameters: {'loss_function': 'MultiClassOneVsAll', 'learning_rate': 0.0006063637820612392, 'l2_leaf_reg': 0.07795528793867033, 'colsample_bylevel': 0.023744337708824698, 'depth': 8, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'min_data_in_leaf': 19, 'one_hot_max_size': 18, 'subsample': 0.3260135756125067}. Best is trial 0 with value: 55.0.
[I 2022-03-03 13:15:51,350] Trial 2 finish

Number of completed trials: 10
Best trial:
	Best Score: 61.0
	Best Params: 
    loss_function: MultiClass
    learning_rate: 0.34728185800513134
    l2_leaf_reg: 0.04036637126917636
    colsample_bylevel: 0.019727335880744393
    depth: 1
    boosting_type: Plain
    bootstrap_type: Bernoulli
    min_data_in_leaf: 12
    one_hot_max_size: 5
    subsample: 0.6835430029680867


In [84]:
cat_model_hdl = CatBoostClassifier(loss_function = 'MultiClass',
                                   learning_rate = 4.871583569813239e-05,l2_leaf_reg = 0.03057672463527161,
                                   colsample_bylevel = 0.05558935393998754,depth = 9, boosting_type = "Plain",
                                   bootstrap_type = "Bayesian", min_data_in_leaf = 16, one_hot_max_size=5, bagging_temperature = 2.790707081889993,)

In [85]:
# Cholesterol hdl
cat_hdl = cat_model_hdl.fit(df_train,y_hdl_cholesterol_human)
hdl_cholesterol_human = cat_hdl.predict(X_test)
test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
test['hdl_cholesterol_human'].value_counts()

0:	learn: 1.0985919	total: 30.7ms	remaining: 30.7s
1:	learn: 1.0985714	total: 69.9ms	remaining: 34.9s
2:	learn: 1.0985484	total: 110ms	remaining: 36.5s
3:	learn: 1.0985257	total: 155ms	remaining: 38.7s
4:	learn: 1.0985078	total: 188ms	remaining: 37.5s
5:	learn: 1.0984883	total: 234ms	remaining: 38.8s
6:	learn: 1.0984661	total: 273ms	remaining: 38.7s
7:	learn: 1.0984408	total: 314ms	remaining: 39s
8:	learn: 1.0984198	total: 355ms	remaining: 39.1s
9:	learn: 1.0983957	total: 410ms	remaining: 40.6s
10:	learn: 1.0983703	total: 459ms	remaining: 41.3s
11:	learn: 1.0983535	total: 509ms	remaining: 41.9s
12:	learn: 1.0983345	total: 541ms	remaining: 41s
13:	learn: 1.0983161	total: 570ms	remaining: 40.2s
14:	learn: 1.0982946	total: 631ms	remaining: 41.4s
15:	learn: 1.0982682	total: 673ms	remaining: 41.4s
16:	learn: 1.0982456	total: 720ms	remaining: 41.7s
17:	learn: 1.0982243	total: 764ms	remaining: 41.7s
18:	learn: 1.0982051	total: 809ms	remaining: 41.8s
19:	learn: 1.0981852	total: 862ms	remaining

ok    61
Name: hdl_cholesterol_human, dtype: int64

In [1]:
# Heamoglobin
X = df_train
y = y_hemoglobin_hgb_human

study = optuna.create_study(sampler=TPESampler(), direction="maximize")
study.optimize(objective, n_trials=10, timeout=600) # Run for 10 minutes
print("Number of completed trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial

print("\tBest Score: {}".format(trial.value))
print("\tBest Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

NameError: ignored

In [87]:
cat_model_hgb = CatBoostClassifier(loss_function = 'MultiClass',
                                   learning_rate = 0.000831198094698226, l2_leaf_reg = 0.16969396366857056,
                                   colsample_bylevel = 0.0903639295440102, depth = 10, boosting_type = "Plain",
                                   bootstrap_type = "Bayesian", min_data_in_leaf = 7, one_hot_max_size = 20,
                                   subsample = 8.74231700445664,)

In [89]:
# # Heamoglobin
cat_hgb = cat_model_hgb.fit(df_train,y_hemoglobin_hgb_human)
hemoglobin_hgb_human = cat_hgb.predict(X_test)
test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
test['hemoglobin_hgb_human'].value_counts()

CatBoostError: ignored

In [ ]:
# from catboost import CatBoostClassifier, Pool
# def cat_model_boost(x_train,y_train):
#     cat_model = CatBoostClassifier(
#     iterations=1000,
#     loss_function='MultiClass',
#     bootstrap_type='Bayesian',
#     eval_metric='MultiClass',
#     leaf_estimation_iterations=100,
#     random_strength=0.5,
#     depth=7,
#     l2_leaf_reg=5,
#     learning_rate=0.01,
#     bagging_temperature=0.5,
#     task_type=None,
# )
#     # cross validation function to find the best parameters
#     scores = []
#     for i in range(10):
#         scores.append(cross_val_score(cat_model,x_train,y_train,cv=5))
#     print(scores)
#     print("The 10 times 10 fold accuracy of the catboost classifier is: ",np.average(scores))   
#     return cat_model


In [44]:
# Cholesterol hdl
# cat_hdl = cat_model_boost(df_train,y_hdl_cholesterol_human)
# hdl_cholesterol_human = cat_hdl.predict(X_test)
# test['hdl_cholesterol_human'] = hdl_cholesterol_human.astype(str)
# test['hdl_cholesterol_human'] = test['hdl_cholesterol_human'].replace(reverse_mapping)
# test['hdl_cholesterol_human'].value_counts()

In [54]:
# # Heamoglobin
# cat_hgb = cat_model_boost(df_train,y_hdl_cholesterol_human)
# hemoglobin_hgb_human = cat_hgb.predict(X_test)
# test['hemoglobin_hgb_human'] = hemoglobin_hgb_human.astype(str)
# test['hemoglobin_hgb_human'] = test['hemoglobin_hgb_human'].replace(reverse_mapping)
# test['hemoglobin_hgb_human'].value_counts()

In [58]:
pwd

'/content'

Catboost with scaled data gave a score of 0.46, with a learning rat of 0.01, it improved the score to 0.49

#### Submission

In [59]:
#maindir = "/Users/Abimbola/Documents/MSc Computer Science/Semester 2/Computer science project/Blood-Spectorscopy/Data" # Directory with your files
maindir = "/content"
ss =  maindir+"/Updated_Sample_Submission.csv"
ss = pd.read_csv(ss)
ss.head()

ss = ss.drop(['Unnamed: 0'], axis= 1)

In [60]:
def transform_c_hdl(row):
    return pd.Series([str(row["donation_id"]) + "_hdl_cholesterol_human", row['hdl_cholesterol_human']])

In [61]:
sample_c_hdl = pd.DataFrame(columns = ss.columns)

In [62]:
from tqdm.notebook import tqdm

tqdm.pandas()
sample_c_hdl[['Donation_ID', 'target']] = test.progress_apply(transform_c_hdl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [63]:
def transform_hemo(row):
    return pd.Series([str(row["donation_id"]) + "_hemoglobin(hgb)_human", row["hemoglobin_hgb_human"]])

In [64]:
sample_hemo = pd.DataFrame(columns = ss.columns)

In [65]:
sample_hemo[['Donation_ID', 'target']] = test.progress_apply(transform_hemo, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [66]:
sample_c_hdl

,Donation_ID,target
355,ID_1961_hdl_cholesterol_human,ok
356,ID_2067_hdl_cholesterol_human,ok
357,ID_2155_hdl_cholesterol_human,ok
358,ID_2211_hdl_cholesterol_human,ok
359,ID_2327_hdl_cholesterol_human,high
...,...,...
411,ID_6373_hdl_cholesterol_human,ok
412,ID_6381_hdl_cholesterol_human,ok
413,ID_6391_hdl_cholesterol_human,ok
414,ID_7755_hdl_cholesterol_human,ok


In [67]:
def transform_c_ldl(row):
    return pd.Series([str(row["donation_id"]) + "_cholesterol_ldl_human", row["cholesterol_ldl_human"]])

In [68]:
sample_c_ldl = pd.DataFrame(columns = ss.columns)

In [69]:
sample_c_ldl[['Donation_ID', 'target']] = test.progress_apply(transform_c_ldl, axis = 1)

  0%|          | 0/61 [00:00<?, ?it/s]

In [70]:
submission = pd.concat([sample_c_hdl, sample_hemo, sample_c_ldl]).reset_index(drop=True)

In [71]:
submission = submission.loc[:, ~submission.columns.str.contains('^Unnamed')]

In [72]:
submission.head()

,Donation_ID,target
0,ID_1961_hdl_cholesterol_human,ok
1,ID_2067_hdl_cholesterol_human,ok
2,ID_2155_hdl_cholesterol_human,ok
3,ID_2211_hdl_cholesterol_human,ok
4,ID_2327_hdl_cholesterol_human,high


In [73]:
submission.to_csv("./submission17.csv", index=False)

#### To do
 - Bagging with random undersampling
 - Use boosting algorithms (AdaBoost etc)
 - Read up SMOTE for imbalanced classification